# AI-medicine 연습

In [5]:
from dotenv import load_dotenv
from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_community.chains.pebblo_retrieval.enforcement_filters import PINECONE
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_pinecone import PineconeVectorStore

load_dotenv()

True

## 데이터 로드 / 병합
- 의약품  `medicine_docs`
- 의약외품 `sanitary_aid_docs`
- 총 Document 개수: 56786

In [2]:
from langchain.schema import Document
import pandas as pd
import os

# !pip install openpyxl

# 데이터를 저장할 리스트
medicine_docs = []
sanitary_aid_docs = []

# 엑셀 파일 경로
medicine_info_paths = ['../data/medicine_info_0.xlsx', '../data/medicine_info_1.xlsx', '../data/medicine_info_2.xlsx', '../data/medicine_info_3.xlsx']
sanitary_aid_info_paths = ['../data/sanitary_aid_info_0.xlsx', '../data/sanitary_aid_info_1.xlsx']

# 파일 경로 체크
print("파일 리스트:", medicine_info_paths, sanitary_aid_info_paths)
for path in medicine_info_paths + sanitary_aid_info_paths:
    print(f"Checking path: {path}")
    print(f"Exists: {os.path.exists(path)}")


# 엑셀 파일 리드 >> Document 변환
for path in medicine_info_paths:
    df = pd.read_excel(path) 
    # 각 행을 Document로 변환
    for _, row in df.iterrows():
        content = row.to_dict() # 행 데이터 >> dict
        medicine_docs.append(Document(page_content=str(content)))

for path in sanitary_aid_info_paths:
    df = pd.read_excel(path)
    for _, row in df.iterrows():
        content = row.to_dict()
        sanitary_aid_docs.append(Document(page_content=str(content)))

# 결과 확인
print(f"의약품 Document 개수: {len(medicine_docs)}")
print(f"의약외품 Document 개수: {len(sanitary_aid_docs)}")
print(f"의약품 첫 번째 Document 내용: {medicine_docs[0].page_content}")
print(f"의약외품 첫 번째 Document 내용: {sanitary_aid_docs[0].page_content}")


파일 리스트: ['../data/medicine_info_0.xlsx', '../data/medicine_info_1.xlsx', '../data/medicine_info_2.xlsx', '../data/medicine_info_3.xlsx'] ['../data/sanitary_aid_info_0.xlsx', '../data/sanitary_aid_info_1.xlsx']
Checking path: ../data/medicine_info_0.xlsx
Exists: True
Checking path: ../data/medicine_info_1.xlsx
Exists: True
Checking path: ../data/medicine_info_2.xlsx
Exists: True
Checking path: ../data/medicine_info_3.xlsx
Exists: True
Checking path: ../data/sanitary_aid_info_0.xlsx
Exists: True
Checking path: ../data/sanitary_aid_info_1.xlsx
Exists: True
의약품 Document 개수: 36983
의약외품 Document 개수: 19803
의약품 첫 번째 Document 내용: {'제품명': '(주)창신의료용산소', '제품영문명': 'Chang-Shin Medical Oxygen', '업체명': '(주)창신', '업체영문명': 'Chang-Shin Co.,LTD', '품목구분': '의약품', '주성분': '산소', '첨가제': nan, '전문의약품': '전문의약품', '완제/원료': '완제의약품', '마약구분': nan, '모양': nan, '색상': nan, '제형': nan, '장축': nan, '단축': nan, '주성분영문': 'Oxygen'}
의약외품 첫 번째 Document 내용: {'제품명': '#4500제이라스틱', '업체명': '(주)곰스포츠', '업체영문명': nan, '품목구분': '의약외품', '주성분': n

In [3]:
documents = medicine_docs + sanitary_aid_docs
print(len(documents))

56786


## 임베딩 모델 생성

: RAG를 구축하기 위한 임베딩 모델 생성
* 문서 내용을 벡터화하여 검색 및 검색된 정보를 바탕으로 답변 생성에 사용

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩 모델 생성
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [4]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# gpt 4o-mini 답변 (document 적용X)
def medicine_info(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''당신은 전문 약사입니다. 사용자가 약물 성분에 대해 질문하면, 아래의 기준에 따라 친절하고 신뢰성 있는 답변을 제공합니다. 질문에 따라 적절한 세부 정보를 포함하되, 지나치게 복잡한 전문 용어는 쉽게 풀어서 설명합니다. 필요한 경우 사용자에게 추가로 주의 사항을 알려줍니다.

    1. 성분명 및 기본 정보:
    - 해당 성분의 이름(화학명 및 일반명)을 정확히 설명하세요.
    - 이 성분이 포함된 대표적인 약물을 예로 들어 설명합니다.

    2. 효능/효과:
    - 이 성분이 치료 또는 완화에 사용되는 질환/증상을 설명합니다.
    - 일반적인 용도를 간결하게 안내합니다.

    3. 작용 기전:
    - 이 성분이 신체에서 어떻게 작용하는지 간단히 설명합니다.
    - 너무 복잡한 메커니즘 대신, 쉽게 이해할 수 있는 표현을 사용하세요.

    4. 부작용:
    - 일반적인 부작용(발생 가능성이 높은 증상)을 명시하세요.
    - 드물지만 심각한 부작용도 언급하며, 발생 시 대처 방법을 알려줍니다.

    5. 복용 및 주의사항:
    - 올바른 복용 방법(용량, 시간 등)을 안내합니다.
    - 음식, 다른 약물, 알코올 등과의 상호작용 여부를 명확히 설명합니다.
    - 임신, 수유 중 복용 가능 여부를 포함합니다.

    6. 상호작용:
    - 이 성분이 특정 약물이나 음식과 상호작용할 가능성이 있는 경우를 설명합니다.

    7. 금기 사항:
    - 이 성분을 복용해서는 안 되는 특정 조건(알레르기, 특정 질환 등)을 명시합니다.

    8. 추가 권장 사항:
    - 사용자가 혼란스러울 수 있는 부분에 대해 명확히 하고, 필요시 의사나 약사와 상담을 권합니다.


    예시 프롬프트
    질문: "이부프로펜 성분에 대해 알려주세요."

    답변:  
    "안녕하세요, 저는 약사입니다. 이부프로펜에 대해 설명드릴게요.  
    1. 성분명: 이부프로펜은 비스테로이드성 소염진통제(NSAID)로, 흔히 사용하는 진통제 및 소염제입니다.  
    2. 효능/효과: 주로 통증 완화(두통, 치통, 생리통), 염증 완화, 발열 감소에 사용됩니다.  
    3. 작용 기전: 이부프로펜은 체내 염증 매개 물질인 프로스타글란딘의 생성을 억제하여 통증과 염증을 줄입니다.  
    4. 부작용 위장 장애(속 쓰림, 위염), 드물게 위궤양, 피부 발진 등이 발생할 수 있습니다.  
    5. 복용 및 주의사항 식사 후에 복용하는 것이 좋으며, 공복에 복용하면 위장 장애를 유발할 수 있습니다.  
    6. 상호작용: 아스피린, 와파린 같은 항응고제와 함께 복용하면 출혈 위험이 증가할 수 있으니 주의하세요.  
    7. 금기 사항: 소화성 궤양이 있거나 NSAID에 과민반응을 보인 적이 있는 분은 복용하지 마세요.  
    8. 추가 권장 사항: 복용 중 이상 증상이 나타나면 즉시 의사나 약사와 상담하세요."

    유의사항
        - 약물 성분 정보는 최신 지침을 바탕으로 작성되어야 하며, 신뢰할 수 있는 데이터베이스(예: Micromedex, UpToDate, 약학정보원 등)를 활용해 근거를 제공해야 합니다.
        - 사용자의 질문을 명확히 파악하여, 필요 이상으로 과도한 정보는 생략하고 핵심만 전달합니다.

        '''),
        ('user', '이미지의 약에 대해 설명해줘'),
    ])
    prompt += HumanMessagePromptTemplate.from_template([query])
    model = ChatOpenAI(
        model='gpt-4o',
        temperature=1
    )
    
    output_parser = StrOutputParser()
    
    chain = prompt | model | output_parser
    return chain

In [6]:
# prompt 확인
chain = RunnableLambda(medicine_info)
chain.invoke({'image_url': 'https://kdpharma.co.kr/uploaded/shop_good/143/0bdcbaa6376b40dbe9c5e10c0e3b26010.jpg'})

'이 사진에 있는 약은 "에이서"로, 주 성분은 아세클로페낙입니다. 아세클로페낙은 비스테로이드성 소염진통제(NSAID)로 여러 통증과 염증을 완화하는 데 사용됩니다.\n\n1. **성분명 및 기본 정보**: \n   - 아세클로페낙(Aceclofenac)은 주로 관절염이나 근육통과 같은 염증성 질환에 처방됩니다.\n\n2. **효능/효과**: \n   - 소염 진통제로, 관절염, 류마티스성 질환, 근육통 등의 통증과 염증을 완화합니다.\n\n3. **작용 기전**: \n   - 이 약물은 프로스타글란딘이라는 염증 매개체의 생성을 억제하여 통증과 염증을 줄입니다.\n\n4. **부작용**: \n   - 위장 장애(소화불량, 속 쓰림), 구역, 발진 등이 있을 수 있습니다. 드물게 위궤양이나 간 기능 이상이 나타날 수 있으므로 주의해야 합니다.\n\n5. **복용 및 주의사항**: \n   - 의사의 처방에 따라 복용합니다. 식사 후 복용하면 위장 장애를 줄일 수 있습니다.\n\n6. **상호작용**: \n   - 아스피린이나 다른 NSAID, 항응고제와 함께 사용 시 주의가 필요합니다.\n\n7. **금기 사항**: \n   - 소화성 궤양 병력이 있거나 NSAID에 민감한 분들은 복용을 피해야 합니다.\n\n추가로 이상 증상이 나타나면 의사나 약사와 상담하는 것이 좋습니다.'

## Pinecone Vector DB 클라이언트 생성
- 임베딩 벡터를 벡터 저장소에 저장하고, 이를 효율적으로 검색하기 위해 사용
- Pinecone은 벡터 검색을 위한 클라우드 기반 서비스 / 매우 빠르고 효율적으로 벡터 저장 & 검색 시스템

In [ ]:
!pip install langchain-pinecone

In [5]:
# batch로 documents 밀어넣기
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')


def batch_documents(documents, batch_size=5000):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i+batch_size]
        
batch_size = 5000
for batch in batch_documents(documents, batch_size):
    PineconeVectorStore.from_documents(
        batch,
        embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key = PINECONE_API_KEY
    )

In [ ]:
os.getenv('PINECONE_INDEX_NAME')

## Pinecone 유사도 검색

In [8]:
# 클라이언트 객체

vector_db = PineconeVectorStore(
    embedding = embeddings,
    index_name=PINECONE_INDEX_NAME,
    namespace=PINECONE_NAMESPACE,
    pinecone_api_key = PINECONE_API_KEY
)

In [18]:
# 약 주성분 검색 함수

from langchain_core.runnables import RunnableLambda

def similarity_search_medicine(query):
    results = vector_db.similarity_search(
        query,
        k=3,
        namespace=PINECONE_NAMESPACE
    )
    
    if not results:
        return {
            "medicine_name": query,
            "medicine_ingredients": "No matching medicine found."
        }
    
    # 결과에서 주성분 추출
    medicine_ingredients = []
    for doc in results:
        # `metadata`에서 약물 관련 정보 가져오기
        doc_metadata = doc.metadata  # 문서의 메타데이터
        main_ingredient = doc_metadata.get("주성분", "Unknown")  # 주성분 정보
        medicine_name = doc_metadata.get("제품명", "Unknown")  # 제품명 정보
        
        # 주성분 정보 저장
        medicine_ingredients.append(f"Product: {medicine_name}, Main Ingredient: {main_ingredient}")
    
    return {
        "medicine_name": query,
        "medicine_ingredients": '\n'.join(medicine_ingredients)
    }

In [20]:
chain = RunnableLambda(similarity_search_medicine)
result = chain.invoke('힙스브이파워정의 주성분 알려줘.')
print(result)

{'medicine_name': '힙스브이파워정의 주성분 알려줘.', 'medicine_ingredients': 'Product: Unknown, Main Ingredient: Unknown\nProduct: Unknown, Main Ingredient: Unknown\nProduct: Unknown, Main Ingredient: Unknown'}


## 서비스 프롬프트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

### 1. 약물 복용 가이드 서비스
**목적**: 사용자가 현재 복용 중인 약물에 대한 **안전한 복용 가이드라인**을 제공.

**주요 기능**:

- **복용 중인 약물 안전성 검토**:
    - 사용자가 입력한 약물 리스트의 성분 분석.
    - 성분 간 **상호작용 경고** 및 **부작용 안내**.
- **복용 가이드라인 제공**:
    - 연령, 건강 상태(임신, 노인 등) 기반의 권장 용량.
    - 복용 시간 간격 및 식사 여부 관련 주의사항.


In [21]:
def guide_medicine(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        당신은 전문 약사입니다. 사용자가 약물 성분에 대해 질문하면, 아래의 기준에 따라 친절하고 신뢰성 있는 답변을 제공합니다. 질문에 따라 적절한 세부 정보를 포함하되, 지나치게 복잡한 전문 용어는 쉽게 풀어서 설명합니다. 필요한 경우 사용자에게 추가로 주의 사항을 알려줍니다.
        '''),
        ('user', '제가 현재 여러 개의 약을 동시에 복용 중인데, 제대로 먹고 있는지 확인하고 싶어요.')
    ])
    
    prompt += HumanMessagePromptTemplate.from_messages([query])
    model = ChatOpenAI(model='gpt-4o', temperature = 0)
    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    
    return chain

### 2. 의약품 / 의약외품 추천 서비스 (made in korea)
**목적**: 사용자의 **개인 맞춤형 의약품** 또는 의약외품을 추천.

**주요 기능**:

- **사용자 입력**: 증상, 선호 성분(포함/제외), 연령, 건강 상태.
- **맞춤 추천**:
    - 입력된 조건에 부합하는 **한국산 의약품/의약외품** 추천.
    - 증상 완화에 적합한 **대체 의약품** 옵션 제시.
- **제품 비교**:
    - 추천 제품 간 주요 성분 및 가격 비교.
    - 사용 후기 및 평판 기반의 제품 선택 지원.

In [ ]:
def recommend_medicine(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        당신은 전문 약사입니다. 사용자가 약물 성분에 대해 질문하면, 아래의 기준에 따라 친절하고 신뢰성 있는 답변을 제공합니다. 질문에 따라 적절한 세부 정보를 포함하되, 지나치게 복잡한 전문 용어는 쉽게 풀어서 설명합니다. 필요한 경우 사용자에게 추가로 주의 사항을 알려줍니다.
        '''),
        # ('user', '임산부인데, 감기에 걸렸어요. 안전하게 먹을 수 있는 약 추천해 주세요.')
    ])

### 3. 약물 안전성 검증 서비스
**목적**: 새로운 약물 또는 의약외품 복용 전, 안전성을 확인.

**주요 기능**:

- **약물 입력 및 검증**:
    - 새롭게 복용하려는 약물의 성분 분석.
    - 현재 복용 중인 약물과의 **상호작용 위험** 평가.
- **부작용 및 위험 경고**:
    - 성분 간 부작용 가능성.
    - 사용자 건강 상태(임신, 만성질환 등)에 따른 **주의사항**.
- **안전 대체제 추천**:
    - 위험성 경고 시, 더 안전한 대체 의약품 제안.


In [ ]:
def check_medicine(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        당신은 전문 약사입니다. 사용자가 약물 성분에 대해 질문하면, 아래의 기준에 따라 친절하고 신뢰성 있는 답변을 제공합니다. 질문에 따라 적절한 세부 정보를 포함하되, 지나치게 복잡한 전문 용어는 쉽게 풀어서 설명합니다. 필요한 경우 사용자에게 추가로 주의 사항을 알려줍니다.
        '''),
        # ('user', '생리통 진통제 A를 복용 중인데, 두통약 E를 추가로 먹어도 될까요?')
    ])